In [2]:
import xarray

In [4]:
data = xarray.open_dataset('S2S_Data/output.grib', engine='cfgrib')
data.to_netcdf('S2S_Data/output.nc')
data

<xarray.Dataset>
Dimensions:     (time: 31, step: 185, latitude: 121, longitude: 240)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-31
  * step        (step) timedelta64[ns] 0 days 00:00:00 ... 46 days 00:00:00
    surface     float64 ...
  * latitude    (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude   (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    tp          (time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-19T14:09 GRIB to CDM+CF via cfgrib-0.9.1...